In [28]:
import setup
setup.init_django()

In [29]:
from market.models import StockQuote

In [30]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [31]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

# filtering data
qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

4334

In [32]:
total = 0
for obj in qs:
    total += obj.close_price

In [33]:
total /qs.count()

Decimal('219.3161923396400553760959852')

In [34]:
# average prices for entire month
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 2, 'avg_price': Decimal('170.1696036020583190')}, {'company': 1, 'avg_price': Decimal('226.9548117035457211')}]>

In [35]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker = ticker).order_by('time')[:count]) # grabbing orginal window
rolling_qs.reverse() # orginal time order

In [36]:
rolling_qs

[<StockQuote: StockQuote object (313492)>,
 <StockQuote: StockQuote object (313491)>,
 <StockQuote: StockQuote object (313490)>,
 <StockQuote: StockQuote object (313489)>,
 <StockQuote: StockQuote object (313488)>]

In [37]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 313492 234.1600 234.1600
2 313491 234.3650 234.2625
3 313490 234.2950 234.2733333333333333333333333
4 313489 234.3950 234.30375
5 313488 234.4400 234.3310


In [38]:
latest_daily_timestamps = (
    StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date - timedelta(days=40), end_date))
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date') # skipping over dates not in database
)

# timestamps for final of the day
actual_timestamps = [x['latest_time'] for x in latest_daily_timestamps]

In [39]:
qs = StockQuote.objects.filter(
    company__ticker = ticker, 
    time__range=(start_date, end_date),
    time__in=actual_timestamps
)

In [40]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma = Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=frame_start, end=0), # based on amount we want to get
        )
).order_by('-time')

In [41]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

330238 235.5300 231.8323400000000000 2024-11-27 00:55:00+00:00
330226 235.4617 230.4683400000000000 2024-11-26 23:55:00+00:00
330053 229.9500 228.9760000000000000 2024-11-23 00:55:00+00:00
330046 229.9899 228.5560000000000000 2024-11-22 23:55:00+00:00
329863 228.2301 227.6100200000000000 2024-11-21 23:55:00+00:00


In [42]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5 = Window( # 5 minute intervals of 5
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-4, end=0), # based on amount we want to get
        ),
        ma_20 = Window( # 20 minute intervals of 5
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-19, end=0), # based on amount we want to get
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(330238,
 Decimal('235.5300'),
 Decimal('231.8323400000000000'),
 Decimal('227.1750850000000000'))